In [2]:
# ---- Step 1: Import Libraries ----
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import gradio as gr

# ---- Step 2: Load and Prepare Data ----
# Load your training data (make sure the file is in your notebook's directory)
train = pd.read_csv(r"C:\Users\esusa\Downloads\Data-Driven Fever Diagnosis Using Machine Learning\trainn.csv")

# Keep only Dengue and Malaria
train = train[train['prognosis'].isin(['Dengue', 'Malaria'])]

# Features and target
feature_columns = train.columns[:-1]
X = train[feature_columns]
y = train['prognosis']

# ---- Step 3: Train the Model ----
model = RandomForestClassifier()
model.fit(X, y)

# ---- Step 4: Define Prediction Function with Confidence ----
def predict_disease_with_confidence(*symptoms):
    # Convert Yes/No to 1/0
    binary_input = [1 if val == "Yes" else 0 for val in symptoms]
    input_array = np.array(binary_input).reshape(1, -1)

    # Predict probabilities
    probs = model.predict_proba(input_array)[0]
    classes = model.classes_

    # Identify top prediction
    top_index = np.argmax(probs)
    predicted = classes[top_index]
    confidence = probs[top_index] * 100

    # Format output
    result = f"🩺 Predicted Disease: {predicted.upper()}\n\n"
    result += f"📊 Confidence: {confidence:.2f}%\n\n"
    result += "🔍 Probabilities:\n"
    for cls, prob in zip(classes, probs):
        result += f" - {cls}: {prob * 100:.2f}%\n"

    return result

# ---- Step 5: Gradio Interface ----
inputs = [
    gr.Radio(choices=["No", "Yes"], value="No", label=feature.replace("_", " ").title())
    for feature in feature_columns
]

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 🧬 Dengue vs Malaria Predictor")
    gr.Markdown("Select the symptoms below and click **Predict Disease**.")

    with gr.Row():
        with gr.Column():
            inputs_block = [inp.render() for inp in inputs]
        with gr.Column():
            output = gr.Textbox(label="Diagnosis", interactive=False, lines=6)

    predict_btn = gr.Button("Predict Disease", elem_id="blue-btn")

    predict_btn.click(predict_disease_with_confidence, inputs=inputs, outputs=output)

    # Custom blue button CSS
    gr.HTML("""
    <style>
    #blue-btn {
        background-color: #007BFF;
        color: white;
        border: none;
        padding: 10px 20px;
        font-weight: bold;
        font-size: 16px;
        margin-top: 10px;
        cursor: pointer;
    }
    </style>
    """)

# ---- Step 6: Launch App ----
demo.launch(share=False)


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
